In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from specim_test.specim.specfuncs import spec2d
from astropy.io import fits as pf

In [2]:
file_loc = '../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/*'

In [4]:
file_list = []
out_file = []
for i,p in enumerate(glob.glob(file_loc)):
    if 'spec2d' in p and 'J0248+1913' in p:
        file_list.append(p)
        frame = p.split('_')[-3][:4]
        out_file.append('J0248_r'+ frame + '_cleaned.fits')

['../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0037-J0248+1913_LRISr_2020Nov21T095645.974.fits',
 '../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0039-J0248+1913_LRISr_2020Nov21T112410.282.fits',
 '../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0038-J0248+1913_LRISr_2020Nov21T104024.845.fits']

In [6]:
## median image
spec_list = []
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[30, 220])
    spec.set_dispaxis('y')
    spec.subtract_sky_2d(use_skymod=True)
    spec_list.append(spec.data)
median_im = np.median(np.asarray(spec_list), axis=0)


Loading file ../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0037-J0248+1913_LRISr_2020Nov21T095645.974.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0037-J0248+1913_LRISr_2020Nov21T095645.974.fits (HDU=12)
The input dataset was trimmed
 xrange: 30:221.  yrange: 0:4096
Final data dimensions (x y): 191 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 191


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



pypeit generated sky model will be used for sky subtraction


From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


Loading file ../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0039-J0248+1913_LRISr_2020Nov21T112410.28

In [7]:
pf.PrimaryHDU(median_im).writeto('J0248_median_image.fits')